# FurnaGui program

Autor: Rafael C. F. Mendes

Laboratório de Energia e Ambiente

Programa para desennho de geometria de turbina para Simulações em CFD

Este programa funciona com a entrada de dois arquivos

- 1- Arquivo de pontos do perfil do aerofólio da turbina (disponível em: http://airfoiltools.com/)
 
- 2- Aquivo fonte da pá. Um arquivo de pontos contendo três colunas de dados (Raio, corda e Torção)

        a) Raio: Coordenada radial para posicionamento do perfil (unidade: metros)
        b) Corda: Tamanho do aerofólio em no raio definido anteriormente (unidade: metros)
        c) Torção: Ângulo de torção do perfil no raio definido anteriormente (unidade: radianos)


In [40]:
#Libraries 
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import subprocess
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
from scipy.interpolate import splprep, splev
import stl
from stl import mesh
import numpy as np
import os
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from PIL import Image, ImageTk
import trimesh
import trimesh.viewer
import trimesh.transformations as tf

#
class App(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("TurbineGenerator UNB-FURNAS")
        self.master.configure(bg="#ffffff")   

      
        photo = []      
        
        image = Image.open("turbineGUIheaderRound.png")
        # resize image
        new_size = (469, 171)  # set new size here
        image = image.resize(new_size)   
        photo = ImageTk.PhotoImage(image)
        # Create a Label widget to display the image
        header = tk.Label(self.master, image=photo)
        header.image = photo  # Keep a reference to the PhotoImage to prevent it from being garbage collected
        header.grid(row=0, column=0, columnspan=3, rowspan=5)
        

        self.file1_path = tk.StringVar(value='NACA653618.dat')
        self.file2_path = tk.StringVar(value='shapeTUCUNARE.dat')
        self.num_blades = tk.IntVar(value=3)
        self.towerHeight = tk.DoubleVar(value=15)
        self.towerRadius = tk.DoubleVar(value=0.5)  
        self.foundationHeight = tk.DoubleVar(value=2)
        self.foundationRadius = tk.DoubleVar(value=5)  
        self.output = tk.StringVar(value="turbine")
        
        #Creates the entries 
         
       
        tk.Label(self.master, text="Airfoil:",font=("Helvetica", 12, "bold"), bg="#ffffff").grid(row=0, column=3)
        tk.Entry(self.master, textvariable=self.file1_path, width=50).grid(row=0, column=4)
        tk.Button(self.master, text="Browse", command=self.browse_file1,font=("Helvetica", 12)).grid(row=0, column=5)
    
        tk.Label(self.master, text="Shape:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=1, column=3)
        tk.Entry(self.master, textvariable=self.file2_path, width=50).grid(row=1, column=4)
        tk.Button(self.master, text="Browse", command=self.browse_file2,font=("Helvetica", 12)).grid(row=1, column=5)
        
        tk.Label(self.master, text="Number of Blades:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=2, column=3)
        tk.Entry(self.master, textvariable=self.num_blades, width=10).grid(row=2, column=4)
        
        tk.Label(self.master, text="Tower height [m]:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=3, column=3)
        tk.Entry(self.master, textvariable=self.towerHeight, width=10).grid(row=3, column=4)
        
        tk.Label(self.master, text="Tower radius [m]:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=4, column=3)
        tk.Entry(self.master, textvariable=self.towerRadius, width=10).grid(row=4, column=4)
        
        tk.Label(self.master, text="Foundation height [m]:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=5, column=3)
        tk.Entry(self.master, textvariable=self.foundationHeight, width=10).grid(row=5, column=4)
        
        tk.Label(self.master, text="Foundation radius [m]:",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=6, column=3)
        tk.Entry(self.master, textvariable=self.foundationRadius, width=10).grid(row=6, column=4)                 
        
        tk.Label(self.master, text="Output File",font=("Helvetica", 12, "bold"),bg="#ffffff").grid(row=7, column=3)
        tk.Entry(self.master, textvariable=self.output).grid(row=7, column=4)
        
 
        tk.Button(self.master, text="Generate Turbine!", command=self.run_script, width=50,
                  fg="black", bg="green",font=("Helvetica", 12, "bold")).grid(row=8, column=3,columnspan=2)
        
        
    def browse_file1(self):
        self.file1_path.set(filedialog.askopenfilename())
    def browse_file2(self):
        self.file2_path.set(filedialog.askopenfilename())
               


        
    def run_script(self):
        file1_path = self.file1_path.get()
        file2_path = self.file2_path.get()
        

        # Inputs for the rotor Design: airfoil, blade shape and number of blades
        pontos = np.loadtxt(file1_path)
        curva = np.loadtxt(file2_path)      
        NumberOfBlades= self.num_blades.get()

        # Inputs for tower design
        towerHeight= self.towerHeight.get()
        towerRadius= self.towerRadius.get()
        
        # Inputs for foundation desing
        foundationHeight= self.foundationHeight.get()
        foundationRadius= self.foundationRadius.get()
        
        
        z = curva[:,0]
        corda = curva[:,1]
        alpha = curva[:,2] * -1

        # Define hub Radius to reach the first data from shapeFile
        h_cylinder=corda[0]*np.sin(alpha[0])*(-1)

        ###########################################
        #              Tower Design
        ########################################### 
        
        r = towerRadius
        npoints = 100 # define the number of points at the tower section profile
        theta = np.linspace(0, 2*np.pi, npoints)

        # Define circle points at beginning
        circle1 = np.zeros((npoints, 3))
        circle1[:, 0] = r*np.cos(theta)
        circle1[:, 1] = r*np.sin(theta)+h_cylinder
        circle1[:, 2] = 0

        # Define circle points in the end
        circle2 = np.zeros((npoints, 3))
        circle2[:, 0] = r*np.cos(theta)
        circle2[:, 1] = r*np.sin(theta)+h_cylinder
        circle2[:, 2] = -towerHeight

        # Concatenate circles and reshape
        vertices = np.concatenate([circle1, circle2]).reshape(-1, 3)

        # Create faces
        faces = []
        for i in range(npoints-1):
            faces.append([i, i+1, i+npoints+1])
            faces.append([i, i+npoints+1, i+npoints])

        # Connect last point to first point
        faces.append([npoints-1, 0, npoints])
        faces.append([npoints-1, npoints, 2*npoints-1])

        # Save solid in STL format
        solid = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
        for k, f in enumerate(faces):
            for m in range(3):
                solid.vectors[k][m] = vertices[f[m]]

        solid.save("tower_C.stl")
        outputFilename=self.output.get()
        outputFilenamesave = outputFilename + '_tower.stl'
        solid.save(outputFilenamesave)
        

        ###########################################
        #              Foundation Design
        ########################################### 
        r = foundationRadius
        npoints = 100
        theta = np.linspace(0, 2*np.pi, npoints)

        # Define circle points at beginning
        circle1 = np.zeros((npoints, 3))
        circle1[:, 0] = r*np.cos(theta)
        circle1[:, 1] = r*np.sin(theta)+h_cylinder
        circle1[:, 2] = -towerHeight

        circle2 = np.zeros((npoints, 3))
        circle2[:, 0] = r*np.cos(theta)
        circle2[:, 1] = r*np.sin(theta)+h_cylinder
        circle2[:, 2] = -towerHeight -foundationHeight

        # Concatenate circles and reshape
        vertices = np.concatenate([circle1, circle2]).reshape(-1, 3)

        # Create faces
        faces = []
        for i in range(npoints-1):
            faces.append([i, i+1, i+npoints+1])
            faces.append([i, i+npoints+1, i+npoints])

        # Connect last point to first point
        faces.append([npoints-1, 0, npoints])
        faces.append([npoints-1, npoints, 2*npoints-1])

        # Save solid in STL format
        solid = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
        for k, f in enumerate(faces):
            for m in range(3):
                solid.vectors[k][m] = vertices[f[m]]
        solid.save("foundation.stl")   
        
        
        ##
        vertices = []
        faces = []

        num_points = 100
        theta = np.linspace(0, 2*np.pi, num_points)
        vertices = np.zeros((num_points, 3))
        vertices[:, 0] = r * np.cos(theta)
        vertices[:, 2] = -towerHeight
        vertices[:, 1] = r * np.sin(theta)

        # Define the faces of the circle
        faces = np.zeros((num_points-2, 3), dtype=np.uint32)
        for i in range(num_points-2):
            faces[i] = [0, i+1, i+2]

        # Create the mesh object
        circle_mesh = stl.mesh.Mesh(np.zeros(faces.shape[0], dtype=stl.mesh.Mesh.dtype))
        for i, f in enumerate(faces):
            for j in range(3):
                circle_mesh.vectors[i][j] = vertices[f[j],:]

        # Save the mesh to an STL file
        circle_mesh.save('circle_tf.stl')
        ##
        
                ##
        vertices = []
        faces = []

        num_points = 100
        theta = np.linspace(0, 2*np.pi, num_points)
        vertices = np.zeros((num_points, 3))
        vertices[:, 0] = r * np.cos(theta)
        vertices[:, 2] = -towerHeight-foundationHeight
        vertices[:, 1] = r * np.sin(theta)

        # Define the faces of the circle
        faces = np.zeros((num_points-2, 3), dtype=np.uint32)
        for i in range(num_points-2):
            faces[i] = [0, i+1, i+2]

        # Create the mesh object
        circle_mesh = stl.mesh.Mesh(np.zeros(faces.shape[0], dtype=stl.mesh.Mesh.dtype))
        for i, f in enumerate(faces):
            for j in range(3):
                circle_mesh.vectors[i][j] = vertices[f[j],:]

        # Save the mesh to an STL file
        circle_mesh.save('circle_bf.stl')
        ##
        
        filenames=[]
        filenames.append('circle_tf.stl')
        filenames.append('circle_bf.stl')
        filenames.append('foundation.stl')

        combined_mesh = stl.mesh.Mesh(np.zeros(0, dtype=stl.mesh.Mesh.dtype))

        # Loop over filenames
        for filename in filenames:
            # Load current mesh
            mesh = stl.mesh.Mesh.from_file(filename)
            # Combine current mesh with previous meshes
            combined_mesh = stl.mesh.Mesh(np.concatenate([combined_mesh.data, mesh.data]))
        
        # Save combined mesh to file

        outputFilenamesave = outputFilename + '_foundation.stl'
        combined_mesh.save(outputFilenamesave)
        
        
        n, _ = curva.shape

        x=np.zeros((n, 200))
        y=np.zeros((n, 200))
        z_n=np.zeros((n, 200))

        for i in range(n):
            perfil = pontos.copy()
            perfil = perfil * corda[i]
            perfil[:,0] = perfil[:,0] - corda[i]/3

            M = np.array([[np.cos(alpha[i]), np.sin(alpha[i])],
                          [-np.sin(alpha[i]), np.cos(alpha[i])]])

            perfil_new = M.dot(perfil.T).T
            perfil_n = np.column_stack((perfil_new, np.ones_like(perfil_new[:,0])*z[i]))

            # Create spline for the lower surface
            tck_upper, u_upper = splprep([perfil_n[:,0], perfil_n[:,1], perfil_n[:,2]], s=0, per=True)    

            # Evaluate splines
            u_new = np.linspace(0, 1, 200)
            x_new_upper, y_new_upper, z_new_upper = splev(u_new, tck_upper)

            x[i,:]=x_new_upper
            y[i,:]=y_new_upper
            z_n[i,:]=z_new_upper


        #my_list.insert(0, 1)

        for j in range(n-1):

             # Create lofted solid
            vertices = []
            faces = []

            for i in range(len(x_new_upper)):
                vertices.append([x[j,i], y[j,i], z_n[j,i]])
                vertices.append([x[j+1,i], y[j+1,i], z_n[j+1,i]])
                if i > 0:
                    faces.append([2*i-2, 2*i-1, 2*i])
                    faces.append([2*i-1, 2*i+1, 2*i])

            # Save solid in STL format
            solid = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
            for k, f in enumerate(faces):
                for m in range(3):
                    solid.vectors[k][m] = vertices[f[m]]

            filename = f"airfoil_perfil_{j}.stl"
            solid.save(filename)

        #Close the last profile
        # Create vertices and faces for the STL mesh
        vertices = []
        faces = []

        _, num_points = x.shape
        vertices = np.zeros((num_points, 3)) 
        vertices[:, 0] = x[n-1,:]
        vertices[:, 1] = y[n-1,:]
        vertices[:, 2] = z_n[n-1,:]

        # Define the faces of the circle
        faces = np.zeros((num_points-2, 3), dtype=np.uint32)
        for i in range(num_points-2):
            faces[i] = [0, i+1, i+2]

        # Create the mesh object
        circle_mesh = stl.mesh.Mesh(np.zeros(faces.shape[0], dtype=stl.mesh.Mesh.dtype))
        for i, f in enumerate(faces):
            for j in range(3):
                circle_mesh.vectors[i][j] = vertices[f[j],:]

        # Save the mesh to an STL file
        circle_mesh.save('last_profile.stl')

        #mesh_object.save('spline.stl')

        filenames=[]

        for j in range(n-1):
            filename = f"airfoil_perfil_{j}.stl"
            filenames.append(filename)

        #filename = 'spline.stl' #last one
        #filenames.append(filename)


        combined_mesh = stl.mesh.Mesh(np.zeros(0, dtype=stl.mesh.Mesh.dtype))

        # Loop over filenames
        for filename in filenames:
            # Load current mesh
            mesh = stl.mesh.Mesh.from_file(filename)

            # Combine current mesh with previous meshes
            combined_mesh = stl.mesh.Mesh(np.concatenate([combined_mesh.data, mesh.data]))

        # Save combined mesh to file
        combined_mesh.save("combined_airfoils.stl")


        for j in range(n-1):
            filename = f"airfoil_perfil_{j}.stl"
            # Delete the temp airfoil files
            os.remove(filename)

        # Define circle parameters
        r = z_n[1,1]
        npoints = 100

        theta = np.linspace(0, 2*np.pi, npoints)

        # Define circle points
        circle1 = np.zeros((npoints, 3))
        circle1[:, 0] = r*np.cos(theta)
        circle1[:, 2] = r*np.sin(theta)
        circle1[:, 1] = h_cylinder

        circle2 = np.zeros((npoints, 3))
        circle2[:, 0] = r*np.cos(theta)
        circle2[:, 2] = r*np.sin(theta)
        circle2[:, 1] = -h_cylinder

        # Concatenate circles and reshape
        vertices = np.concatenate([circle1, circle2]).reshape(-1, 3)

        # Create faces
        faces = []
        for i in range(npoints-1):
            faces.append([i, i+1, i+npoints+1])
            faces.append([i, i+npoints+1, i+npoints])

        # Connect last point to first point
        faces.append([npoints-1, 0, npoints])
        faces.append([npoints-1, npoints, 2*npoints-1])

        # Save solid in STL format
        solid = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
        for k, f in enumerate(faces):
            for m in range(3):
                solid.vectors[k][m] = vertices[f[m]]

        solid.save("hub.stl")

        #tampa do cilindro back
        #from stl import mesh


        # Define the parameters of the circle
        radius = r
        num_points = npoints

        # Create the vertex array for the circle
        theta = np.linspace(0, 2*np.pi, num_points)
        vertices = np.zeros((num_points, 3))
        vertices[:, 1] = h_cylinder
        vertices[:, 0] = radius * np.cos(theta)
        vertices[:, 2] = radius * np.sin(theta)

        # Define the faces of the circle
        faces = np.zeros((num_points-2, 3), dtype=np.uint32)
        for i in range(num_points-2):
            faces[i] = [0, i+1, i+2]

        # Create the mesh object
        circle_mesh = stl.mesh.Mesh(np.zeros(faces.shape[0], dtype=stl.mesh.Mesh.dtype))
        for i, f in enumerate(faces):
            for j in range(3):
                circle_mesh.vectors[i][j] = vertices[f[j],:]

        # Save the mesh to an STL file
        circle_mesh.save('circle_b.stl')

        # Set the radius of the sphere
        radius = r

        # Define the number of points to use in each dimension
        num_points = 50

        # Generate the vertices and faces of the sphere
        phi = np.linspace(0, np.pi, num_points)
        theta = np.linspace(np.pi,2*np.pi, num_points)
        phi, theta = np.meshgrid(phi, theta)
        x = radius * np.sin(phi) * np.cos(theta)
        y = radius * np.sin(phi) * np.sin(theta)
        z = radius * np.cos(phi)

        # Create a list of vertices and faces
        vertices = np.vstack([x.flatten(), y.flatten(), z.flatten()]).T
        faces = []
        for i in range(num_points-1):
            for j in range(num_points-1):
                p1 = i * num_points + j
                p2 = p1 + 1
                p3 = p1 + num_points
                p4 = p3 + 1
                faces.append([p1, p2, p3])
                faces.append([p2, p4, p3])

        # Translate the sphere to the desired origin
        vertices += [0.0, -h_cylinder, 0.0]

        # Create a mesh object and add the vertices and faces to it
        sphere_mesh = stl.mesh.Mesh(np.zeros(len(faces), dtype=stl.mesh.Mesh.dtype))
        for i, f in enumerate(faces):
            for j in range(3):
                sphere_mesh.vectors[i][j] = vertices[f[j]]

        # Save the mesh as an STL file
        sphere_mesh.save('sphere.stl')

        # Load the STL file
        mesh = stl.mesh.Mesh.from_file("combined_airfoils.stl")

        # Define the rotation axis and angle
        axis = [0.0, 1.0, 0.0 ]


        for i in range(NumberOfBlades):
            angle = np.pi *2 / NumberOfBlades 
            # Rotate the mesh around the z-axis by angle degrees
            mesh.rotate(axis, angle)
            filename = f"blade_number_{i+1}.stl"
            mesh.save(filename)

        filenames=[]

        for j in range(NumberOfBlades):
            filename = f"blade_number_{j+1}.stl"
            filenames.append(filename)

        filenames.append("hub.stl")
        filenames.append('sphere.stl')
        filenames.append('circle_b.stl')
        filenames.append('last_profile.stl')

        filenames.append("tower_C.stl")
        
        filenames.append('circle_tf.stl')
        filenames.append('circle_bf.stl')
        filenames.append("foundation.stl")

        combined_mesh = stl.mesh.Mesh(np.zeros(0, dtype=stl.mesh.Mesh.dtype))

        # Loop over filenames
        for filename in filenames:
            # Load current mesh
            mesh = stl.mesh.Mesh.from_file(filename)
            # Combine current mesh with previous meshes
            combined_mesh = stl.mesh.Mesh(np.concatenate([combined_mesh.data, mesh.data]))

        # Save combined mesh to file
        outputFilename=self.output.get()
        outputFilenamesave = outputFilename + '_complete.stl'
        combined_mesh.save(outputFilenamesave)
        
        

        for j in range(NumberOfBlades):
            filename = f"blade_number_{j+1}.stl"
            # Delete the temp airfoil files
            os.remove(filename)
        os.remove("hub.stl")
        os.remove('sphere.stl')
        os.remove('circle_b.stl')    
        os.remove('last_profile.stl')
        os.remove('circle_tf.stl')
        os.remove('circle_bf.stl')
        os.remove('foundation.stl')
        os.remove("tower_C.stl")
        
        
        self.fig = Figure(figsize=(4, 3), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.fig, self.master)
        self.canvas.get_tk_widget().grid(row=9, column=3, columnspan=2, rowspan=2)
        
        
        mesh = trimesh.load(outputFilenamesave)
        self.fig.clear()
        ax = self.fig.add_subplot(111, projection='3d')
        ax.plot_trisurf(mesh.vertices[:,0], mesh.vertices[:,1], mesh.vertices[:,2], triangles=mesh.faces)#, edgecolor="k",linewidth=0.01)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        min_y = np.min(mesh.vertices[:, 1])
        max_y = np.max(mesh.vertices[:, 1])
        # Remove blank space
        tight_layout = {'pad': 0.1}
        self.fig.tight_layout(rect=[0, 0, 1, 1], **tight_layout)
        #step_size = max(3, (max_y - min_y) // 10)  # choose step size of at least 3, or 1/10 of range if that's larger
       # y_values = np.arange(min_y, max_y, step_size)
        ax.yaxis.set_ticks([min_y, max_y])

        ax.set_zlabel('Z')
        ax.set_aspect('equal')
        self.canvas.draw()
        
        self.fig1 = Figure(figsize=(4, 1), dpi=100)
        self.canvas = FigureCanvasTkAgg(self.fig1, self.master)
        self.canvas.get_tk_widget().grid(row=8, column=1, columnspan=2)
        
        ax1 = self.fig1.add_subplot()
        #fig1, ax1 = plt.subplots()
        ax1.plot(pontos[:,0],pontos[:,1])
        ax1.set_aspect('equal')
        ax1.set_title("Airfoil plot")

        #canvas = tkagg.FigureCanvasTkAgg(fig1, master=root)
        #canvas.draw()
        #canvas.get_tk_widget().grid(row=5, column=3, columnspan=2)


        
        # Create a Treeview widget to display the table
        self.table = ttk.Treeview(self.master, columns=("R", "Chord", "Twist")
                                  , show="headings")
        self.table.heading("R", text="R[m]")
        self.table.heading("Chord", text="Chord[m]")
        self.table.heading("Twist", text="Twist[rad]")
        self.table.column("R", width=50)
        self.table.column("Chord", width=60)
        self.table.column("Twist", width=70)

        # Insert the data into the table
       
        for row in curva:
            formatted_row = [f"{val:.2f}" for val in row]
            self.table.insert("", "end", values=tuple(formatted_row), tags=("centered",))


            #self.table.insert("", "end", values=tuple(formatted_row))
 
       # Display the table using the grid method
        self.table.grid(row=9, column=0, columnspan=3)


root = tk.Tk()
app = App(master=root)
app.grid()
app.mainloop()
